### Import Packages

In [86]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.metrics import r2_score
from scipy.stats.mstats import winsorize
from sklearn.linear_model import Lasso, Ridge

from utils.system import *
from class_data.data import *

import warnings
warnings.filterwarnings('ignore')

### Load Data

In [69]:
# Load dataset
prisk = pd.read_stata(get_data() / 'prisk_replication' / 'RestrictedReplication_FirmQuarter.dta')
prisk.gvkey = prisk.gvkey.astype('int64')
prisk_gvkey = prisk.gvkey.unique().tolist()
prisk['date'] = pd.to_datetime(prisk['cdateQ']).dt.to_period('Q').dt.to_timestamp('Q')
prisk = prisk.set_index(['gvkey', 'date'])
prisk = prisk.sort_index(level=['gvkey', 'date'])
print(f"Length of PRisk GVKEY: {len(prisk_gvkey)}")
print(f"Data Columns: \n{prisk.columns.tolist()}")

Length of PRisk GVKEY: 9478
Data Columns: 
['cdateQ', 'Risk', 'PRisk', 'NPRisk', 'Sentiment', 'PSentiment', 'MISSINGsic', 'MISSINGsic2', 'MISSINGsue', 'MISSINGd2at', 'MISSINGlat', 'llobnewF', 'donation_total_nrF', 'hedgegroupF', 'ldonF', 'lcontractamount', 'PRiskMDA', 'Risk_std', 'PRisk_std', 'NPRisk_std', 'Sentiment_std', 'PSentiment_std', 'PRiskMDA_std', 'firm_id', 'aPRisk_std', 'PRisk_std_d2at', 'beta2_aPRisk_std', 'beta_aPRisk_std', 'beta_PRisk_aPRisk_std', 'beta2_PRisk_aPRisk_std', 'lcontractamount_aPRisk_std', 'MISSINGimpvol_w_std', 'MISSINGvolatility_w_std', 'capital_investm_w_100', 'MISSINGdeltasales_w_100', 'MISSINGpct_capex_guidance1_w_100', 'MISSINGav_retW_100']


In [4]:
# Load JKP daily ret
daily_ret = Data(folder_path=get_format_data() / 'jkp', file_pattern='daily_ret_*')
daily_ret = daily_ret.concat_files()
daily_ret.shape

Loading Data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.02it/s]


(219492890, 6)

In [5]:
# Load JKP daily ret
characteristics = Data(folder_path=get_format_data() / 'jkp', file_pattern='characteristics_*')
characteristics = characteristics.concat_files()
characteristics = characteristics.dropna(subset='gvkey')
characteristics['gvkey'] = characteristics['gvkey'].astype('int64')
characteristics.shape

Loading Data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  5.60it/s]


(3394206, 444)

### Format Data

In [35]:
# Create asset dataframe
asset = characteristics[['id', 'date', 'gvkey', 'assets']]
asset['date'] = pd.to_datetime(asset['date'], format='%Y%m%d')

In [22]:
# Create id and gvkey map to join JKP datasets
id_gvkey = characteristics[['id', 'gvkey']]
id_gvkey = id_gvkey.drop_duplicates(subset='gvkey', keep='last')
id_to_gvkey_mapping = id_gvkey.set_index('id')['gvkey'].to_dict()

In [24]:
# Add gvkey to JKP dataset
daily_ret['gvkey'] = daily_ret['id'].map(id_to_gvkey_mapping)
daily_ret = daily_ret.dropna(subset='gvkey')
daily_ret['gvkey'] = daily_ret['gvkey'].astype('int64')

In [37]:
# Retrieve GVKEY data used in index
daily_ret = daily_ret.loc[daily_ret.gvkey.isin(prisk_gvkey)]
asset = asset.loc[asset.gvkey.isin(prisk_gvkey)]

In [41]:
# Change daily_ret date to datetime
daily_ret['date'] = pd.to_datetime(daily_ret['date'], format='%Y%m%d')

In [44]:
# Set index
daily_ret = daily_ret.set_index(['gvkey', 'date'])
asset = asset.set_index(['gvkey', 'date'])

In [55]:
# Calculate quarterly volatility
vol_q = daily_ret.groupby('gvkey')[['ret']].resample('Q', level='date').std()
vol_q.columns = ['vol']

In [75]:
# Get quarterly asset
asset_q = asset.groupby('gvkey')[['assets']].resample('Q', level='date').last()

In [88]:
# Join everything
regress = prisk.join(vol_q)
regress = regress.join(asset_q)
regress = regress[['PRisk', 'vol', 'assets']]
regress = regress.dropna()

### Regress

In [93]:
# Winsorize
regress['log_assets'] = np.log(regress['assets'])
regress['log_assets'] = winsorize(regress['log_assets'], limits=[0.01, 0.01])
regress['PRisk'] = winsorize(regress['PRisk'], limits=[0.01, 0.01])
regress['vol'] = winsorize(regress['vol'], limits=[0.01, 0.01])

In [94]:
# Standardize (divide by the standard deviation)
std_devs = regress[['PRisk', 'log_assets', 'vol']].std()
regress['PRisk_scaled'] = regress['PRisk'] / std_devs['PRisk']
regress['log_assets_scaled'] = regress['log_assets'] / std_devs['log_assets']
regress['vol_scaled'] = regress['vol'] / std_devs['vol']
regress = regress.dropna()

In [98]:
# Alpha
X = sm.add_constant(regress[['PRisk_scaled', 'log_assets_scaled']])
y = regress['vol_scaled']

# Create the regression model
model = sm.OLS(y, X)
results = model.fit()

# Print out the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             vol_scaled   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                 1.293e+04
Date:                Thu, 02 May 2024   Prob (F-statistic):               0.00
Time:                        00:56:46   Log-Likelihood:            -1.8879e+05
No. Observations:              141418   AIC:                         3.776e+05
Df Residuals:                  141415   BIC:                         3.776e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 3.0136      0.00